In [1]:
from test_utils import *
from swayam import Agent

### Function Calling with Swayam Tools

Let's now start to link the functionality of an LLM Agent with a locally available function call.

#### Let's see what happens without a function call

In [2]:
from swayam import Swayam
Swayam.execute("What's the weather like in Boston?")

Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
What's the weather like in Boston?
--------------------------------------------------------------------------------
Response:
I don't have real-time data access to provide current weather conditions. To find out the weather in Boston, I recommend checking a reliable weather website or app for the most up-to-date information.
--------------------------------------------------------------------------------


ChatCompletionMessage(content="I don't have real-time data access to provide current weather conditions. To find out the weather in Boston, I recommend checking a reliable weather website or app for the most up-to-date information.", refusal=None, role='assistant', function_call=None, tool_calls=None)

### Some Dummy Functions

Let's create some dummy functions

In [3]:
from enum import Enum
class UnitEnum(Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit):
    import random
    i = random.randint(-10, 40)
    if unit.lower() == "celsius":
        return f"{i}°C"
    elif unit.lower() == "fahrenheit":
        return f"{i*9/5 + 32}°F"
    else:
        raise ValueError("Invalid unit")
    
def calculate(expression):
    return eval(expression)

def search_email(text):
    import re
    emails = re.findall(r'([\w\.-]+@[\w\.-]+)', text)
    if emails:
        return emails
    else:
        return None

## Tool Creation

As a part of an LLM workflow, we want to trigger the above local function based on response from the LLM (using function calling).

Before we can do that, we need to create a sort of bridge between a function from the view point of an LLM vs what it means in Python.

Here we see how we can create convert a function into a Tool.

### ToolBuilder

We use the tool builder to wrap the function created above.
1. Provide the function callable and describe it.
2. Add all arguments, while describing their types and purposes.

In [4]:
from swayam.llm.tool import ToolBuilder
builder = ToolBuilder(get_current_weather, desc="Get the current weather in a given location")
builder.add_field("location", type=str, desc="Location to get weather information for")
builder.add_field("unit", type=UnitEnum, desc="Unit of temperature", default=UnitEnum.CELSIUS)
get_current_weather_tool = builder.build()

builder = ToolBuilder(calculate, desc="Evaluates a mathematical expression")
builder.add_field("expression", type=str, desc="The expression to be evaluated.")
calc_tool = builder.build()

builder = ToolBuilder(search_email, desc="Searches email addresses in a given text")
builder.add_field("text", type=str, desc="The target text in which emails are to be searched.")
search_email_tool = builder.build()


### Tool Execution

The key thing to keep in mind is that all arguments are passed as keyword arguments irrespective of the underlying function.

In [5]:
get_current_weather_tool(location="dummy")

'30°C'

In [6]:
get_current_weather_tool(**{"location": "dummy"})

'36°C'

In [7]:
get_current_weather_tool(**{"location": "dummy", "unit":"fahrenheit"})

'48.2°F'

In [8]:
calc_tool(expression="2 + 5")

7

In [9]:
search_email_tool(text="This is a sample text with a@b.com address")

['a@b.com']

In [10]:
search_email_tool(text="This is a sample text with no email address")

### Tool Definition

One of the key benefits of creating a Tool is that it self describes itself, in a way compatible with an LLM.

In [11]:
print(get_current_weather_tool.definition)

{'name': 'get_current_weather', 'description': 'Get the current weather in a given location', 'parameters': {'properties': {'location': {'description': 'Location to get weather information for', 'title': 'Location', 'type': 'string'}, 'unit': {'description': 'Unit of temperature', 'enum': ['celsius', 'fahrenheit'], 'type': 'string'}}, 'required': ['location'], 'type': 'object'}}


In [12]:
print(calc_tool.definition)

{'name': 'calculate', 'description': 'Evaluates a mathematical expression', 'parameters': {'properties': {'expression': {'description': 'The expression to be evaluated.', 'title': 'Expression', 'type': 'string'}}, 'required': ['expression'], 'type': 'object'}}


In [13]:
print(search_email_tool.definition)

{'name': 'search_email', 'description': 'Searches email addresses in a given text', 'parameters': {'properties': {'text': {'description': 'The target text in which emails are to be searched.', 'title': 'Text', 'type': 'string'}}, 'required': ['text'], 'type': 'object'}}


Let's create the list of function definitions to be shared with the LLM.

In [14]:
functions = [get_current_weather_tool.definition, calc_tool.definition, search_email_tool.definition]

Let's create a tools dictionary so that once we know the name of the tool from LLM, we can call it with arguments returned from the LLM.

In [15]:
tools = {
    get_current_weather_tool.name : get_current_weather_tool,
    calc_tool.name : calc_tool,
    search_email_tool.name : search_email_tool
}

In [16]:
from swayam import Agent
agent = Agent(display=True)

In [17]:
function_json = agent.execute("What's the weather like in Boston?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))

Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
What's the weather like in Boston?
--------------------------------------------------------------------------------
Response:
None
--------------------------------------------------------------------------------
FunctionCall(arguments='{"location":"Boston"}', name='get_current_weather')


'22°C'

In [18]:
function_json = agent.execute("What is 7 *30 + 13?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))

Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
What is 7 *30 + 13?
--------------------------------------------------------------------------------
Response:
None
--------------------------------------------------------------------------------
FunctionCall(arguments='{"expression":"7 * 30 + 13"}', name='calculate')


223

In [19]:
text = "This is a sample text with a@b.com and c@d.com addresses"
function_json = agent.execute(f"Find whether there is an email address in the following text marked with triple backticks \n ```{text}```?", functions=functions).function_call
tools[function_json.name](**json.loads(function_json.arguments))

Total Context Length (Previous Request-Response Pairs): 0
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
Prompt: (Role: user)
Find whether there is an email address in the following text marked with triple backticks 
 ```This is a sample text with a@b.com and c@d.com addresses```?
--------------------------------------------------------------------------------
Response:
None
--------------------------------------------------------------------------------
FunctionCall(arguments='{"text":"This is a sample text with a@b.com and c@d.com addresses"}', name='search_email')


['a@b.com', 'c@d.com']